In [1]:
from google.colab import drive
from tensorflow import keras
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
import librosa

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive/')


Mounted at /content/drive/


In [5]:
DATA_PATH = './drive/MyDrive/CDTN/songvn'

In [ ]:
def extract_melspectrogram(audio_path, n_mels=128):
    # Load audio file
    y, sr = librosa.load(audio_path, sr=None)

    # Extract Mel-spectrogram
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)

    # Convert to decibels (log scale)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    return mel_spec_db

In [ ]:
parent_folder = './drive/MyDrive/CDTN/songvn'
output_folder = './drive/MyDrive/CDTN/mel_spectrograms'

In [ ]:
import os
import librosa


def extract_features_from_folder(parent_folder, output_folder):
    genres = os.listdir(parent_folder)
    for genre in genres:
        genre_folder = os.path.join(parent_folder, genre)
        if not os.path.isdir(genre_folder):
            continue

        # Tạo thư mục đầu ra cho thể loại hiện tại nếu nó chưa tồn tại
        output_genre_folder = os.path.join(output_folder, genre)
        os.makedirs(output_genre_folder, exist_ok=True)

        # Duyệt qua các tệp âm thanh trong thư mục thể loại hiện tại
        for filename in os.listdir(genre_folder):
            if filename.endswith('.wav'):
                # Đường dẫn đến tệp âm thanh đầy đủ
                audio_path = os.path.join(genre_folder, filename)

                # Trích xuất đặc trưng từ tệp âm thanh
                mel_spec = extract_melspectrogram(audio_path)

                # Lưu đặc trưng vào tệp numpy
                output_filename = os.path.splitext(filename)[0] + '.npy'
                output_file_path = os.path.join(output_genre_folder, output_filename)
                np.save(output_file_path, mel_spec)

def extract_melspectrogram(audio_path, n_mels=128):
    # Load file âm thanh
    y, sr = librosa.load(audio_path, sr=None)

    # Trích xuất Mel-spectrogram
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)

    # Chuyển đổi sang đơn vị dB (log scale)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    return mel_spec_db

# Sử dụng hàm để trích xuất đặc trưng từ thư mục songvn


extract_features_from_folder(parent_folder, output_folder)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Đọc dữ liệu từ các tệp .npy
def load_data_from_npy_folder(folder):
    images = []
    labels = []

    genres = os.listdir(folder)
    class_to_int = {genre: idx for idx, genre in enumerate(genres)}

    for genre in genres:
        genre_folder = os.path.join(folder, genre)
        if not os.path.isdir(genre_folder):
            continue
        class_label = class_to_int[genre]
        for filename in os.listdir(genre_folder):
            if filename.endswith('.npy'):
                file_path = os.path.join(genre_folder, filename)
                data = np.load(file_path)
                images.append(data)
                labels.append(class_label)

    return np.array(images), np.array(labels)

# Chuyển đổi dữ liệu thành hình ảnh PNG và lưu lại
def convert_to_png(images, labels, output_folder):
    unique_labels = np.unique(labels)

    for label in unique_labels:
        label_folder = os.path.join(output_folder, str(label))
        os.makedirs(label_folder, exist_ok=True)

        label_images = images[labels == label]
        for i, image in enumerate(label_images):
            output_path = os.path.join(label_folder, f'image_{i}.png')
            plt.imshow(image, cmap='viridis', origin='lower')
            plt.axis('off')
            plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
            plt.close()

# Đường dẫn đến thư mục chứa các tệp .npy
input_folder_npy = './drive/MyDrive/CDTN/mel_spectrograms'

# Đọc dữ liệu từ các tệp .npy
X_train, y_train = load_data_from_npy_folder(input_folder_npy)

# Đường dẫn đến thư mục chứa hình ảnh PNG đầu ra
output_folder_png = './drive/MyDrive/CDTN/mel_spectrograms_png'

# Chuyển đổi dữ liệu thành hình ảnh PNG và lưu lại
convert_to_png(X_train, y_train, output_folder_png)


In [ ]:
# Normalize data
X_train = X_train / 255.0

In [ ]:
X_train.shape

(15, 128, 782)

In [ ]:
X_train = tf.expand_dims(X_train, axis=-1)

In [ ]:
# Define model architecture
model = tf.keras.models.Sequential([
    # First convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=X_train.shape[1:]),
    tf.keras.layers.MaxPooling2D(2, 2),
    # Second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y_train)), activation='softmax')
])


In [ ]:



# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 1.0753 - accuracy: 0.4167 - val_loss: 60.6752 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 1s 1s/step - loss: 15.4152 - accuracy: 0.4167 - val_loss: 40.9651 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 1s 1s/step - loss: 25.5047 - accuracy: 0.4167 - val_loss: 10.9325 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: 8.5734 - accuracy: 0.4167 - val_loss: 1.3811 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 1s 1s/step - loss: 1.1321 - accuracy: 0.4167 - val_loss: 2.3482 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 1s 1s/step - loss: 1.6995 - accuracy: 0.4167 - val_loss: 1.5020 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 1s 1s/step - loss: 1.2492 - accuracy: 0.4167 - val_loss: 0.9855 - val_accuracy: 0.3333
Epoch 8/10
1

**Phần sửa lại**

In [7]:
import os
import librosa as lb
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

class Preprocessing():
    def __init__(self, data_path, save_path):
        self.data_path = data_path
        self.save_path = save_path
        self.samples = {}

    def _load_samples(self):
        for genre in os.listdir(self.data_path):
            genre_path = os.path.join(self.data_path, genre)
            if os.path.isdir(genre_path):
                for file in os.listdir(genre_path):
                    if file.endswith('.wav'):
                        file_path = os.path.join(genre_path, file)
                        self.samples[file] = {'dir': file_path, 'genre': genre}
        return self.samples

    def _process_samples(self):
        for name, info in self.samples.items():
            file, sr = lb.load(info['dir'])
            self.samples[name]['sampling'] = file
            D = np.abs(lb.stft(file))
            self.samples[name]['stft'] = D
            S = lb.feature.melspectrogram(y=file, sr=sr)
            S_db = lb.amplitude_to_db(S, ref=np.max)
            self.samples[name]['mel-spec-db'] = S_db
        return self.samples

    def _save_mel_spec(self):
        for name, info in self.samples.items():
            S_db = info['mel-spec-db']
            genre = info['genre']
            file_name = os.path.splitext(name)[0]
            save_folder = os.path.join(self.save_path, genre)
            if not os.path.exists(save_folder):
                os.makedirs(save_folder)
                print("Created new folder:", save_folder)
            plt.imsave(os.path.join(save_folder, f"{file_name}.png"), S_db)
            print(f"Saved {file_name}.png")

if __name__ == "__main__":
    DATA_PATH = './drive/MyDrive/CDTN/songvn'
    SAVE_PATH = './drive/MyDrive/CDTN/save_folder'  # Thay đổi đường dẫn tới thư mục lưu hình ảnh mel-spectrogram

    preprocessor = Preprocessing(data_path=DATA_PATH, save_path=SAVE_PATH)
    preprocessor._load_samples()
    preprocessor._process_samples()
    preprocessor._save_mel_spec()


Created new folder: ./drive/MyDrive/CDTN/save_folder/cailuong
Saved shortened_cailuong_5.png
Saved shortened_cailuong_2.png
Saved shortened_cailuong_3.png
Saved shortened_cailuong_4.png
Saved shortened_cailuong_1.png
Created new folder: ./drive/MyDrive/CDTN/save_folder/cheo
Saved shortened_cheo_4.png
Saved shortened_cheo_2.png
Saved shortened_cheo_5.png
Saved shortened_cheo_3.png
Saved shortened_cheo_1.png
Created new folder: ./drive/MyDrive/CDTN/save_folder/quanho
Saved shortened_audio_3.png
Saved shortened_audio_2.png
Saved shortened_audio_5.png
Saved shortened_audio_4.png
Saved shortened_audio_1.png


In [9]:
data_path = './drive/MyDrive/CDTN/save_folder'

In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Tạo generator cho dữ liệu huấn luyện
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Tái tổ chức biểu đồ màu từ [0, 255] sang [0, 1]
    validation_split=0.2  # Phân chia dữ liệu thành tập huấn luyện và tập validation
)

# Tạo generator cho tập huấn luyện
train_generator = train_datagen.flow_from_directory(
    data_path,
    batch_size=32,
    target_size=(128,1292),
    class_mode='categorical',  # Sử dụng one-hot encoding cho nhãn
    subset='training'  # Sử dụng phần dữ liệu huấn luyện
)

# Tạo generator cho tập validation
validation_generator = train_datagen.flow_from_directory(
    data_path,
    batch_size=32,
    target_size=(128,1292),
    class_mode='categorical',
    subset='validation'  # Sử dụng phần dữ liệu validation
)

# Tiếp theo, bạn có thể sử dụng train_generator và validation_generator để huấn luyện mô hình.

Found 12 images belonging to 3 classes.
Found 3 images belonging to 3 classes.


In [22]:
import cv2
# Đường dẫn đến thư mục chứa dữ liệu
data_path = './drive/MyDrive/CDTN/save_folder'

# Lặp qua tất cả các tệp ảnh trong thư mục
for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith('.png'):
                img_path = os.path.join(folder_path, file)
                # Đọc ảnh bằng OpenCV
                img = cv2.imread(img_path)
                # Lấy kích thước của ảnh
                height, width, _ = img.shape
                print(f"Kích thước ảnh '{file}': {width}x{height}")

Kích thước ảnh 'shortened_cailuong_5.png': 2154x128
Kích thước ảnh 'shortened_cailuong_2.png': 2154x128
Kích thước ảnh 'shortened_cailuong_3.png': 2154x128
Kích thước ảnh 'shortened_cailuong_4.png': 2154x128
Kích thước ảnh 'shortened_cailuong_1.png': 2154x128
Kích thước ảnh 'shortened_cheo_4.png': 2154x128
Kích thước ảnh 'shortened_cheo_2.png': 2154x128
Kích thước ảnh 'shortened_cheo_5.png': 2154x128
Kích thước ảnh 'shortened_cheo_3.png': 2154x128
Kích thước ảnh 'shortened_cheo_1.png': 2154x128
Kích thước ảnh 'shortened_audio_3.png': 2154x128
Kích thước ảnh 'shortened_audio_2.png': 2154x128
Kích thước ảnh 'shortened_audio_5.png': 2154x128
Kích thước ảnh 'shortened_audio_4.png': 2154x128
Kích thước ảnh 'shortened_audio_1.png': 2154x128


In [23]:
model1 = tf.keras.models.Sequential([
        #first_convolution
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(128, 1292, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        #second_convolution
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        #third_convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        #fourth_convolution
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

In [24]:
# Compile mô hình
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Sử dụng categorical crossentropy vì ta sử dụng one-hot encoding cho nhãn
              metrics=['accuracy'])

# Huấn luyện mô hình
history = model1.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)


Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 1.0986 - accuracy: 0.3333 - val_loss: 1.1927 - val_accuracy: 0.3333
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 1.1338 - accuracy: 0.3333 - val_loss: 1.3747 - val_accuracy: 0.3333
Epoch 3/10
1/1 [==============================] - 4s 4s/step - loss: 1.3556 - accuracy: 0.3333 - val_loss: 1.1252 - val_accuracy: 0.3333
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 1.1089 - accuracy: 0.3333 - val_loss: 1.1040 - val_accuracy: 0.3333
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 1.0715 - accuracy: 0.6667 - val_loss: 1.1117 - val_accuracy: 0.3333
Epoch 6/10
1/1 [==============================] - 2s 2s/step - loss: 1.0750 - accuracy: 0.3333 - val_loss: 1.1074 - val_accuracy: 0.3333
Epoch 7/10
1/1 [==============================] - 2s 2s/step - loss: 1.0605 - accuracy: 0.3333 - val_loss: 1.0991 - val_accuracy: 0.3333
Epoch 8/10
1/1 [=========================